In [33]:
import os
import glob
import time
import numpy as np
import pickle
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
from skimage.feature import hog
from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label
import model

Using TensorFlow backend.


# Load Images

In [34]:
###
### Load Images
###

basedir = 'vehicles/'
image_types = os.listdir(basedir)
cars = []
for imtype in image_types:
    cars.extend(glob.glob(basedir+imtype+'/*'))
print ('Number of Vehicle Images found: ', len(cars))
with open('cars.txt', 'w') as f:
    for fn in cars:
        f.write(fn + '\n')

basedir = 'non-vehicles/'
image_types = os.listdir(basedir)
notcars = []
for imtype in image_types:
    notcars.extend(glob.glob(basedir+imtype+'/*'))
print ('Number of Non-Vehicle Images found: ', len(notcars))
with open('non-cars.txt', 'w') as f:
    for fn in notcars:
        f.write(fn + '\n')

Number of Vehicle Images found:  8792
Number of Non-Vehicle Images found:  8968


In [35]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

# Feature Extraction

Define helper functions to extract hog, color bins, and color histogram features


In [36]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features


In [37]:
def bin_spatial(img, size=(32, 32)):
    resized = cv2.resize(img, size)
    features = resized.ravel()
    return features

In [38]:
def color_hist(img, nbins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))

    return hist_features

In [39]:
def extract_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True, vis=False):
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(img, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
            hog_features = np.concatenate(hog_features)

        else:
            if vis == True:
                hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel], orient,
                                                            pix_per_cell, cell_per_block, vis=True, feature_vec=True)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                                                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    if vis == True:
        return np.concatenate(img_features), hog_image
    else:
        return np.concatenate(img_features)

In [40]:
def extract_features_from_list(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):

    features = []
    for file in imgs:
        image = mpimg.imread(file)
        file_features = extract_features(image, color_space=color_space,
                            spatial_size=spatial_size, hist_bins=hist_bins,
                            orient=orient, pix_per_cell=pix_per_cell,
                            cell_per_block=cell_per_block,
                            hog_channel=hog_channel, spatial_feat=spatial_feat,
                            hist_feat=hist_feat, hog_feat=hog_feat)
        features.append(file_features)

    return features

# Train Linear SVM Classifier

In [41]:
def train_svc(cars, notcars, color_space, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, hog_channel, spatial_feat, hist_feat, hog_feat):
    t = time.time()
    test_cars = cars
    test_notcars = notcars


    car_features = extract_features_from_list(test_cars, color_space=color_space,
                            spatial_size=spatial_size, hist_bins=hist_bins,
                            orient=orient, pix_per_cell=pix_per_cell,
                            cell_per_block=cell_per_block,
                            hog_channel=hog_channel, spatial_feat=spatial_feat,
                            hist_feat=hist_feat, hog_feat=hog_feat)
    notcar_features = extract_features_from_list(test_notcars, color_space=color_space,
                            spatial_size=spatial_size, hist_bins=hist_bins,
                            orient=orient, pix_per_cell=pix_per_cell,
                            cell_per_block=cell_per_block,
                            hog_channel=hog_channel, spatial_feat=spatial_feat,
                            hist_feat=hist_feat, hog_feat=hog_feat)
    print(time.time()-t, 'Seconds to compute features...')

    X = np.vstack((car_features, notcar_features)).astype(np.float64)

    X_scaler = StandardScaler().fit(X)

    scaled_X = X_scaler.transform(X)

    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    rand_state = np.random.randint(0,100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.1, random_state=rand_state)

    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))

    svc = LinearSVC()

    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    dist_pickle = {}
    dist_pickle["svc"] = svc
    dist_pickle["X_scaler"] = X_scaler
    pickle.dump(dist_pickle, open("./svc_pickle.p", "wb"))

In [20]:
###
### Train SVC
###
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

train_svc(cars, notcars, color_space, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, hog_channel, spatial_feat, hist_feat, hog_feat) 

/Users/BurrusRA/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


156.1359519958496 Seconds to compute features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
29.9 Seconds to train SVC...
Test Accuracy of SVC =  0.991


# Detect Vehicles


In [96]:
class Vehicle_Detector():
    def __init__(self, use_cnn):
        ###
        ### State used for traditional CV method
        ###
        self.color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
        self.orient = 9  # HOG orientations
        self.pix_per_cell = 8 # HOG pixels per cell
        self.cell_per_block = 2 # HOG cells per block
        self.hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
        self.spatial_size = (32, 32) # Spatial binning dimensions
        self.hist_bins = 32    # Number of histogram bins
        self.spatial_feat = True # Spatial features on or off
        self.hist_feat = True # Histogram features on or off
        self.hog_feat = True # HOG features on or off
        self.window_size = 64 
        
        dist_pickle = pickle.load(open("svc_pickle.p", "rb"))
        self.svc = dist_pickle["svc"]
        self.X_scaler = dist_pickle["X_scaler"]
        
        
        self.use_cnn = use_cnn
        self.cnnModel = model.get_model()

        self.cnnModel.load_weights('model_1.h5')
        
        self.heatmaps = []
        
        self.probs = []

        
        
    def draw_boxes(self, img, bboxes, color=(0, 0, 255), thick=6):
        # make a copy of the image
        draw_img = np.copy(img)
        for bbox in bboxes:
            # Draw a rectangle given bbox coordinates
            cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
        # Return the image copy with boxes drawn
        return draw_img

    def draw_labeled_bboxes(self, img, labels):
        #iterate through detected cars
        for car_number in range(1, labels[1]+1):
            #find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()

            #identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            #Define a bouding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        return img

    def apply_threshold(self, heatmap, threshold):
        heatmap[heatmap <= threshold] = 0
        return heatmap
    
    def combined_heatmap(self, heatmap):
        self.heatmaps.append(heatmap)
        heatmap_count = len(self.heatmaps)
        heatmaps = []
        if (heatmap_count < 9):
            heatmaps = self.heatmaps
        else:
            heatmaps = self.heatmaps[(heatmap_count-9):]

        final_heatmap = np.zeros_like(heatmap)
        for hm in heatmaps:
            final_heatmap[hm == 1] += 1
            final_heatmap[hm == 2] += 1
            final_heatmap[hm == 3] += 1
            final_heatmap[hm == 4] += 1
            final_heatmap[hm == 5] += 1
            final_heatmap[hm == 6] += 1
            final_heatmap[hm == 7] += 1
            final_heatmap[hm == 8] += 1
            final_heatmap[hm == 9] += 1
            final_heatmap[hm == 10] += 1
            final_heatmap[hm == 11] += 1
            final_heatmap[hm == 12] += 1
            final_heatmap[hm == 13] += 1
            final_heatmap[hm == 14] += 1
            final_heatmap[hm == 15] += 1
            final_heatmap[hm == 16] += 1
            final_heatmap[hm == 17] += 1
            final_heatmap[hm == 18] += 1
            final_heatmap[hm == 19] += 1
            final_heatmap[hm == 20] += 1
            final_heatmap[hm == 21] += 1
            final_heatmap[hm == 22] += 1


        return final_heatmap

    def detect_vehicles(self, img, draw_img, heatmap, ystart, ystop, scale):

        img = img.astype(np.float32)/255
        xstart = 400 #only consider right side of screen
        img_tosearch = img[ystart:ystop,xstart:,:]
        ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // self.pix_per_cell)-1
        nyblocks = (ch1.shape[0] // self.pix_per_cell)-1
        nfeat_per_block = self.orient*self.cell_per_block**2
        
        nblocks_per_window = (self.window_size // self.pix_per_cell)-1
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        
        
        if not self.use_cnn:
            # Compute individual channel HOG features for the entire image
            hog1 = get_hog_features(ch1, self.orient, self.pix_per_cell, self.cell_per_block, feature_vec=False)
            hog2 = get_hog_features(ch2, self.orient, self.pix_per_cell, self.cell_per_block, feature_vec=False)
            hog3 = get_hog_features(ch3, self.orient, self.pix_per_cell, self.cell_per_block, feature_vec=False)
            

        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                
                
                xleft = xpos*self.pix_per_cell
                ytop = ypos*self.pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+self.window_size, xleft:xleft+self.window_size], (64,64))

                if self.use_cnn:
                    test = np.expand_dims(subimg, axis=0)
                    test_predictions = self.cnnModel.predict(test)
                    test_prediction = test_predictions[0,0,0,0]
                else:
                    # Get color features
                    spatial_features = bin_spatial(subimg, size=self.spatial_size)
                    hist_features = color_hist(subimg, nbins=self.hist_bins)

                    # Extract HOG for this patch
                    hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                    hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                    hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                    hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                    # Scale features and make a prediction
                    test_features = self.X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
                    test_prediction = self.svc.predict(test_features)

                
                self.probs.append(test_prediction)
                
                if test_prediction > 0.5:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(self.window_size*scale)
                    cv2.rectangle(draw_img,(xbox_left+xstart, ytop_draw+ystart),(xbox_left+xstart+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)
                    heatmap[ytop_draw+ystart:ytop_draw+win_draw+ystart, xbox_left+xstart:xbox_left+xstart+win_draw] += 1

        return heatmap, draw_img


    def process_image(self, img):
        
        heatmap = np.zeros_like(img[:,:,0])
        draw_img = np.copy(img)
        
        ###
        ### Detect vehicles at 3 different scales
        ###
        scales = [1, 1.5, 2]
        y_start_stop = [370, 690]
        for scale in scales:
            heatmap, draw_img = self.detect_vehicles(img, draw_img, heatmap, y_start_stop[0], y_start_stop[1], scale)
            
        

        #combine with heatmaps from previous frames
        averaged_heatmap = self.combined_heatmap(heatmap)
        averaged_heatmap = self.apply_threshold(averaged_heatmap, 6)
        labels = label(averaged_heatmap)
        draw_labeled = self.draw_labeled_bboxes(np.copy(img), labels)


        # Overlays draw_img onto draw_labeled to create our final output image 
        x_offset = 50
        y_offset = 350
        l_img = draw_labeled
        s_img = cv2.resize(draw_img, None, fx=.2, fy=.2)
        l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], 0] = s_img[:,:,0]
        l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], 1] = s_img[:,:,1]
        l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], 2] = s_img[:,:,2]

        return l_img

In [100]:
def process_images(fnames):
    vd = Vehicle_Detector(use_cnn=False)
    img_count = len(fnames)
    fig, axs = plt.subplots(img_count, 2, figsize=(20, img_count*5))
    axs = axs.ravel()
    
    for i, fname in enumerate(fnames):
        img = plt.imread(fname)
        img = vd.process_image(img)
        axs[i].imshow(img)
        
        
    plt.show()
    
    print(vd.probs)

In [127]:
images = glob.glob('./video_images/project_video_img814.jpg')
images = images[0:1]
        
#process_images(images)



In [52]:
def process_video():
    test_output = 'project_ouput3.mp4'
    #test_output = 'adv_lane_ouput.mp4'
    clip = VideoFileClip('project_video.mp4')
    #clip = VideoFileClip('adv_lane_video.mp4')
    vd = Vehicle_Detector(use_cnn=False)
    test_clip = clip.fl_image(vd.process_image)
    test_clip.write_videofile(test_output, audio=False)

In [53]:

process_video()

[MoviePy] >>>> Building video project_ouput3.mp4
[MoviePy] Writing video project_ouput3.mp4


  0%|          | 6/1261 [00:37<2:09:02,  6.17s/it]

KeyboardInterrupt: 

In [129]:
#def load_data():
#    with open('data.p', mode='rb') as f:
#        data = pickle.load(f)
#        X_train = data['X_train']
#        X_test = data['X_test']
#        X_val = data['X_val']
#        y_train = data['y_train']
#        y_val = data['y_val']
#        y_test = data['y_test']
#        
#    return np.array(X_train), np.array(X_val), np.array(X_test), np.array(y_train), np.array(y_val), np.array(y_test)
#


In [128]:
#X_train, X_val, X_test, _, _, _ = load_data()
#
#cnnModel = model.get_model()
#cnnModel.load_weights('ppico.h5')
#fnames = X_train[5:20]
#images = []
#for f in fnames:
#    images.append(plt.imread(f))
#
#img_count = len(images)
#fig, axs = plt.subplots(img_count, 4, figsize=(10, img_count*2))
#axs = axs.ravel()
#
#for i, img in enumerate(images):
#    axs[i].imshow(img)
#
#imgs = np.array(images)
##imgs = np.expand_dims(images, axis=0)
#test_predictions = cnnModel.predict(imgs)
#
#print(test_predictions)
#plt.show()